# Module 8: 11 - Anatomy of an AI Agent - Implementing the Agent Executor Tool Class
---------------------------------------------------------------------------------
In this lesson, we will define the AgentToolExecutor Python class to serve as a proxy between the AI Agent and the tools. The AgentToolExecutor class allows the AI Agent to register tools upon initialization and provides an abstraction for executing these tools. It validates the existence of tools, ensures the suggested schema is valid, and uses the AgentTool class to run the tools.

## Objectives
* Define the AgentToolExecutor class to manage tool registration, validation, and execution.
* Ensure tools are correctly registered and accessible by the AI Agent.
* Validate tool suggestions against the registered tool schemas.
* Use the AgentTool class to execute validated tools.

## What this session covers:
* Defining the current agent structure, including the LLM Client and short-term memory.
* Creating the AgentToolExecutor class for tool management.
* Registering tools with the AgentToolExecutor.
* Validating tool suggestions using the registered schemas.
* Executing tools using the AgentTool class and its run method.
* Integrating and testing the AgentToolExecutor class within the AI Agent framework.

## Install Libraries

In [2]:
#! pip install openai

## Define Current Agent Structure

### LLM Client

In [1]:
from typing import Dict, Any, List
import openai

class OpenAIChatCompletion:
    """Interacts with OpenAI's API for chat completions."""
    def __init__(self, model: str, api_key: str = None, base_url: str = None):
        self.client = openai.OpenAI(api_key=api_key, base_url=base_url)
        self.model = model

    def generate(self, messages: List[str], tools: List[Dict[str, Any]] = None, **kwargs) -> Dict[str, Any]:
        """Generates a response from OpenAI's API."""
        params = {'messages': messages, 'model': self.model, 'tools': tools, **kwargs}
        response = self.client.chat.completions.create(**params)
        return response.choices[0].message

### Short-Term Memory

In [2]:
from typing import List, Dict

class ChatMessageMemory:
    """Manages conversation context."""
    
    def __init__(self):
        self.messages = []
    
    def add_message(self, message: Dict):
        """Add a message to memory."""
        self.messages.append(message)
    
    def add_messages(self, messages: List[Dict]):
        """Add multiple messages to memory."""
        for message in messages:
            self.add_message(message)
    
    def get_messages(self) -> List[Dict]:
        """Retrieve all messages."""
        return self.messages.copy()
    
    def reset_memory(self):
        """Clear all messages."""
        self.messages = []

### Agent Tool

In [3]:
from pydantic import BaseModel, ValidationError
from typing import Callable, Type
from inspect import signature

class AgentTool:
    """Encapsulates a Python function with Pydantic validation."""
    def __init__(self, func: Callable, args_model: Type[BaseModel]):
        self.func = func
        self.args_model = args_model
        self.name = func.__name__
        self.description = func.__doc__ or self.args_schema.get('description', '')

    def to_openai_function_call_definition(self) -> dict:
        """Converts the tool to OpenAI Function Calling format."""
        schema_dict = self.args_schema
        description = schema_dict.pop("description", "")
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": description,
                "parameters": schema_dict
            }
        }

    @property
    def args_schema(self) -> dict:
        """Returns the tool's function argument schema as a dictionary."""
        schema = self.args_model.model_json_schema()
        schema.pop("title", None)
        return schema

    def validate_json_args(self, json_string: str) -> bool:
        """Validate JSON string using the Pydantic model."""
        try:
            validated_args = self.args_model.model_validate_json(json_string)
            return isinstance(validated_args, self.args_model)
        except ValidationError:
            return False

    def run(self, *args, **kwargs) -> Any:
        """Execute the function with validated arguments."""
        try:
            # Handle positional arguments by converting them to keyword arguments
            if args:
                sig = signature(self.func)
                arg_names = list(sig.parameters.keys())
                kwargs.update(dict(zip(arg_names, args)))

            # Validate arguments with the provided Pydantic schema
            validated_args = self.args_model(**kwargs)
            return self.func(**validated_args.model_dump())
        except ValidationError as e:
            raise ValueError(f"Argument validation failed for tool '{self.name}': {str(e)}")
        except Exception as e:
            raise ValueError(f"An error occurred during the execution of tool '{self.name}': {str(e)}")

    def __call__(self, *args, **kwargs) -> Any:
        """Allow the AgentTool instance to be called like a regular function."""
        return self.run(*args, **kwargs)

### Agent Tool Decorator

In [4]:
from typing import Callable, Optional, Type
from pydantic import BaseModel

def check_docstring(func: Callable):
    """Ensure the function has a docstring."""
    if not func.__doc__:
        raise ValueError(f"Function '{func.__name__}' must have a docstring.")

def Tool(func: Optional[Callable] = None, *, args_model: Type[BaseModel]) -> AgentTool:
    """Decorator to wrap a function with an AgentTool instance."""
    def decorator(f: Callable) -> AgentTool:
        check_docstring(f)
        return AgentTool(f, args_model=args_model)
    return decorator(func) if func else decorator

### Agent Base

In [5]:
from typing import Dict

class Agent:
    """Integrates LLM client, tools, and memory."""
    def __init__(self, llm_client, system_message: Dict[str, str], tools=None):
        self.llm_client = llm_client
        self.tools = tools
        self.memory = ChatMessageMemory()
        self.system_message = system_message

    def run(self, user_message: Dict[str, str]):
        self.memory.add_message(user_message)
        chat_history = [self.system_message] + self.memory.get_messages()
        response = self.llm_client.generate(chat_history, tools=self.tools)
        self.memory.add_message(response)
        return response

## Implementing The Tool Executor Class

In [31]:
from typing import Any, Dict, List, Optional
import json

class AgentToolExecutor:
    """Manages tool registration and execution."""
    
    def __init__(self, tools: Optional[List[AgentTool]] = None):
        self.tools: Dict[str, AgentTool] = {}
        if tools:
            for tool in tools:
                self.register_tool(tool)
    
    def register_tool(self, tool: AgentTool):
        """Registers a tool."""
        if tool.name in self.tools:
            raise ValueError(f"Tool '{tool.name}' is already registered.")
        self.tools[tool.name] = tool
      
    def execute(self, tool_name: str, tool_args: str) -> Any:
        """Executes a tool by name with given arguments."""
        print(f"Checking if {tool_name} exists..")
        tool = self.tools.get(tool_name)
        if not tool:
            raise ValueError(f"Tool '{tool_name}' not found.")
        try:
            print(f"Validating {tool_name} suggested args {tool_args}")
            if tool.validate_json_args(tool_args):
                tool_args_dict = json.loads(tool_args)
                print(f"Executing {tool_name} with args: {tool_args}")
                return tool.run(**tool_args_dict)
            else:
                raise ValueError(f"Error validating tool '{tool_name}' arguments.")
        except Exception as e:
            raise ValueError(f"Error executing tool '{tool_name}': {e}") from e
    
    def get_tool_names(self) -> List[str]:
        """Returns a list of all registered tool names."""
        return list(self.tools.keys())
    
    def get_tool_details(self) -> str:
        """Returns details of all registered tools."""
        tools_info = [f"{tool.name}: {tool.description} Args schema: {tool.args_schema['properties']}" for tool in self.tools.values()]
        return '\n'.join(tools_info)

## Integrating Tool Executor to Agent

### Updating Agent Base

In [32]:
from typing import Dict

class Agent:
    """Integrates LLM client, tools, and memory."""
    def __init__(self, llm_client, system_message: Dict[str, str], tools=None):
        self.llm_client = llm_client
        self.tools = tools
        self.memory = ChatMessageMemory()
        self.system_message = system_message
        # Adding a Tool Executor
        self.executor = AgentToolExecutor()
        # Adding a Tool History List
        self.tool_history = []
        
        # Register and convert tools
        if tools:
            for tool in tools:
                self.executor.register_tool(tool)
            self.function_calls = [tool.to_openai_function_call_definition() for tool in tools]

    def run(self, user_message: Dict[str, str]):
        self.memory.add_message(user_message)
        chat_history = [self.system_message] + self.memory.get_messages()
        response = self.llm_client.generate(chat_history, tools=self.tools)
        self.memory.add_message(response)
        return response
    
    def run(self, user_message: Dict[str, str]):
        """Generates responses, manages tool calls, and updates memory."""
        self.memory.add_message(user_message)
        chat_history = [self.system_message] + self.memory.get_messages() + self.tool_history
        response = self.llm_client.generate(chat_history, tools=self.function_calls)
        if response.tool_calls:
            for tool in response.tool_calls:
                tool_name = tool.function.name
                tool_args = tool.function.arguments
                try:
                    execution_results = self.executor.execute(tool_name, tool_args)
                    self.tool_history.append({
                        "role": "tool",
                        "tool_call_id": tool.id,
                        "name": tool_name,
                        "content": str(execution_results)
                    })
                except Exception as e:
                    raise ValueError(f"Execution error in tool '{tool_name}': {e}") from e
            return self.tool_history
        else:
            self.memory.add_message(response)
            self.tool_history = []
            return response

## Initialize Client

In [33]:
# API from environment variable
# import os
# api_key = os.getenv("OPENAI_API_KEY"))

api_key=""

client = OpenAIChatCompletion(
    base_url='https://api.openai.com/v1',
    model='gpt-4',
    api_key=api_key
)

## Define System messages

In [34]:
# Define the system message
system_message = {"role": "system", "content": "You are a weather assistant."}

## Define Tools

In [35]:
from pydantic import BaseModel, Field
import random

class GetWeatherSchema(BaseModel):
    """Get weather information based on location."""
    location: str = Field(description="Location to get weather for")

@Tool(args_model=GetWeatherSchema)
def get_weather(location: str) -> str:
    """Gets weather information."""
    temperature = random.randint(60, 80)
    return f"{location}: {temperature}F."

In [36]:
from pydantic import BaseModel, Field

class JumpSchema(BaseModel):
    """Jump a specific distance"""
    distance: str = Field(description="Specific distance to jump for")

@Tool(args_model=JumpSchema)
def jump(distance: str) -> str:
    """Jumps a specific distance."""
    return f"I jumped the following distance {distance}"

In [37]:
tools = [get_weather,jump]

## Initialize Agent

In [38]:
# Initialize the Agent with the LLM client, system message, and tools
agent = Agent(llm_client=client, system_message=system_message, tools=tools)

## Send a User Message

In [39]:
# Define a user message
user_message = {"role": "user", "content": "What is the weather in Virginia?"}

# Generate a response using the agent
response = agent.run(user_message)
response

Checking if get_weather exists..
Validating get_weather suggested args {
  "location": "Virginia"
}
Executing get_weather with args: {
  "location": "Virginia"
}


[{'role': 'tool',
  'tool_call_id': 'call_wi3fRnOAv1t7d6jjuB01KQpr',
  'name': 'get_weather',
  'content': 'Virginia: 63F.'}]